In [1]:
import pandas as pd
import yfinance as yf
from datetime import date
import pandas_ta as ta

In [4]:
def analyze_stock(stock, dados, start, end, window_size):
    # Baixa os dados históricos da ação do Yahoo Finance
    data = yf.download(stock, start=start, end=end)
    
    # Verifica se os dados foram baixados corretamente
    if not data.empty:
        # Reseta o índice para tratar a data como uma coluna
        data.reset_index(inplace=True)
        # Converte a coluna de data para o formato de string 'AAAA-MM-DD'
        dados['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
        # Calcula a variação percentual diária do preço de fechamento ajustado
        dados['Var Percentual'] = data['Adj Close'].pct_change() * 100
        # Calcula a média móvel e o desvio padrão das variações percentuais
        media_rolling = dados['Var Percentual'].rolling(window=window_size).mean()
        desvio_padrao_rolling = dados['Var Percentual'].rolling(window=window_size).std()
        dados['Variancia'] =  dados['Var Percentual'].rolling(window=window_size).var()
        # Calculando a média da variância dos retornos usando a mesma janela
        dados['Media Variancia'] = dados['Variancia'].rolling(window=window_size).mean()
        if isinstance(data, pd.Series):
            data = data.to_frame()
        # Calculando o RSI
        rsi = data.ta.rsi(close='Adj Close', length=14)
        dados['rsi'] = rsi
        dados['Stock'] = stock
        print(type(dados['Date']))
        temp_datas = pd.to_datetime(dados['Date'])
        dados['AnoMes'] = temp_datas.dt.to_period('M')
        print("Dataframe\n {}".format(dados))
        dsv_sup1 = media_rolling + (desvio_padrao_rolling * 1.5)
        dsv_sup2 = media_rolling + (desvio_padrao_rolling * 3)
        dsv_inf1 = media_rolling - (desvio_padrao_rolling * 1.5)
        dsv_inf2 = media_rolling - (desvio_padrao_rolling * 3)
        diferenca = dsv_sup1 - dsv_inf1
        dados['DiferencaDesvios'] = diferenca
        media_diferenca = diferenca.rolling(window=252).mean()
        dados['MediaDiferencaDesvios'] = media_diferenca
        print(media_diferenca)
        dados_filt = dados [(dados['rsi']>= 48)& (dados['rsi']<=52) & (dados['Variancia'] < dados['Media Variancia']) & (dados['Var Percentual']< 4) & (dados['DiferencaDesvios']<dados['MediaDiferencaDesvios'])] 
        dados_unique = dados_filt.drop_duplicates(subset='AnoMes', keep='first')
        dados_unique = dados_unique.drop('AnoMes',axis=1)
        print("Dados únicos")
        print(dados_unique)
        return dados_unique
    else:
        return dados

In [5]:
# Definição de parâmetros iniciais
stocksymbols = ["PETR4.SA", 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC3.SA']
startdate = date(2002, 1, 1)
end_date = date(2023, 12, 31)
window_size = 100

# Processamento de múltiplas ações
all_results = pd.DataFrame()
dados = pd.DataFrame(columns=('Date','AnoMes','Var Percentual','DiferencaDesvios','MediaDiferencaDesvios','rsi','Variancia','Media Variancia','Stock'))
for stock in stocksymbols:
    stock_results = analyze_stock(stock, dados, startdate, end_date, window_size)
    if not stock_results.empty:
        # Combina os resultados de todas as ações em um único DataFrame
        all_results = pd.concat([all_results, stock_results], ignore_index=True)

# Salva os resultados finais em um arquivo JSON
if not all_results.empty:
    all_results.to_json('filtered_straddle_results.json', orient='records')
    print(all_results)
else:
    print("Nenhum dado foi encontrado para as condições especificadas.")

[*********************100%***********************]  1 of 1 completed
<class 'pandas.core.series.Series'>
Dataframe
             Date   AnoMes  Var Percentual DiferencaDesvios  \
0     2002-01-01  2002-01             NaN              NaN   
1     2002-01-02  2002-01        0.195504              NaN   
2     2002-01-03  2002-01        0.682936              NaN   
3     2002-01-04  2002-01       -0.620177              NaN   
4     2002-01-07  2002-01        2.184114              NaN   
...          ...      ...             ...              ...   
5500  2023-12-21  2023-12        0.027479              NaN   
5501  2023-12-22  2023-12        0.961816              NaN   
5502  2023-12-26  2023-12        1.605881              NaN   
5503  2023-12-27  2023-12        0.080360              NaN   
5504  2023-12-28  2023-12       -0.321204              NaN   

     MediaDiferencaDesvios        rsi  Variancia  Media Variancia     Stock  
0                      NaN        NaN        NaN             